# Python Immersion Course

### An Introduction to Python - Part 2

### Joe Blankenship - Just some dude

_
## The Basics
_

### Exercise

For this exercise, we'll be creating a function to scrape a CSV from a website.

* import and use `urllib` library to request and urlopen the CSV of your choice
* import and use the `csv` library to read the csv
* import and use the TextIOWrapper method from the `io` library to parse the CSV data
* use a for loop to print out each row in the CSV

Use the above examples for assistance.

In [ ]:
# place your RPG program here

from urllib.request import urlopen
from io import TextIOWrapper
from csv import reader

csv_url = ''

def csv_scrape(url):
    page = urlopen(url)
    csv_data = reader(TextIOWrapper(page))
    for row in csv_data:
        print(row)

In [ ]:
# test your function here

csv_scrape(sac_csv)

http://www.uky.edu/crimelog/